# 测试stdout输出

你的程序中有个方法会输出到标准输出中（ `sys.stdout` ）。

也就是说它会将文本打印到屏幕上面。 

你想写个测试来证明它，给定一个输入，相应的输出能正常显示出来。

**解决方案**

使用 `unittest.mock` 模块中的 `patch()` 函数， 使用起来非常简单，可以为单个测试模拟 `sys.stdout` 然后回滚， 并且不产生大量的临时变量或在测试用例直接暴露状态变量。

作为一个例子，我们在 mymodule 模块中定义如下一个函数：

In [1]:
%%file mymodule.py

def urlprint(protocol, host, domain):
    url = '{}://{}.{}'.format(protocol, host, domain)
    print(url)

Writing mymodule.py


默认情况下内置的 `print` 函数会将输出发送到 `sys.stdout` 。

为了测试输出真的在那里，你可以使用一个替身对象来模拟它，然后使用断言来确认结果。

使用 `unittest.mock` 模块的 `patch()` 方法可以很方便的在测试运行的上下文中替换对象， 并且当测试完成时候自动返回它们的原有状态。

下面是对 `mymodule` 模块的测试代码：

In [5]:
%%file __init__.py
# for import

Writing __init__.py


In [75]:
%%file mymodule_test.py
from io import StringIO
from unittest import TestCase
from unittest.mock import patch
import mymodule

class TestURLPrint(TestCase):
    def test_url_gets_to_stdout(self):
        protocol = 'http'
        host = 'www'
        domain = 'example.com'
        expected_url = '{}://{}.{}\n'.format(protocol, host, domain)
        with patch('sys.stdout', new=StringIO()) as fake_out:
            mymodule.urlprint(protocol, host, domain)
            self.assertEqual(fake_out.getvalue(), expected_url)



Overwriting mymodule_test.py


In [76]:
!python -m unittest mymodule_test.py

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


`urlprint()` 函数接受三个参数，测试方法开始会先设置每一个参数的值。 

`expected_url` 变量被设置成包含期望的输出的字符串。

`unittest.mock.patch()` 函数被用作一个上下文管理器，使用 `StringIO` 对象来代替 `sys.stdout` . `fake_out` 变量是在该进程中被创建的模拟对象。

在 `with` 语句中使用它可以执行各种检查。

当 `with` 语句结束时， `patch` 会将所有东西恢复到测试开始前的状态

# [unittest.mock.patch()]在单元测试中给对象打补丁

你写的单元测试中需要给指定的对象打补丁， 用来断言它们在测试中的期望行为（比如，断言被调用时的参数个数，访问指定的属性等）

**解决方案**

`unittest.mock.patch()` 函数可被用来解决这个问题。 

`patch()` 还可被用作一个装饰器、上下文管理器或单独使用，尽管并不常见。 

例如，下面是一个将它当做装饰器使用的例子：

```py
from unittest.mock import patch
import example

@patch('example.func')
def test1(x, mock_func):
    example.func(x)       # Uses patched example.func
    mock_func.assert_called_with(x)
```

它还可以被当做一个上下文管理器：

```py
with patch('example.func') as mock_func:
    example.func(x)      # Uses patched example.func
    mock_func.assert_called_with(x)
```    
    
最后，你还可以手动的使用它打补丁：

```py
p = patch('example.func')
mock_func = p.start()
example.func(x)
mock_func.assert_called_with(x)
p.stop()
```

如果可能的话，你能够叠加装饰器和上下文管理器来给多个对象打补丁。例如：

```py
@patch('example.func1')
@patch('example.func2')
@patch('example.func3')
def test1(mock1, mock2, mock3):
    ...

def test2():
    with patch('example.patch1') as mock1, \
         patch('example.patch2') as mock2, \
         patch('example.patch3') as mock3:
    ...
```


`patch()` 接受一个已存在对象的全路径名，将其替换为一个新的值。

原来的值会在装饰器函数或上下文管理器完成后自动恢复回来。 默认情况下，所有值会被 
`MagicMock` 实例替代

In [26]:
x = 42
with patch('__main__.x'):
     print(x)

<MagicMock name='x' id='2680791932488'>


不过，你可以通过给 `patch()` 提供第二个参数来将值替换成任何你想要的：

In [27]:
x = 42
with patch('__main__.x', 'patched_value'):
     print(x)

patched_value


被用来作为替换值的 `MagicMock` 实例能够模拟可调用对象和实例。 

他们记录对象的使用信息并允许你执行断言检查，例如：

In [28]:
from unittest.mock import MagicMock
m = MagicMock(return_value = 10)
m(1, 2, debug=True) # 10
m.assert_called_with(1, 2, debug=True)
m.assert_called_with(1, 2)

10

AssertionError: Expected call: mock(1, 2)
Actual call: mock(1, 2, debug=True)

In [36]:
m.upper.return_value = 'HELLO'
m.upper('hello') # 'HELLO'

assert m.upper.called # True

m.split.return_value = ['hello', 'world']
m.split('hello world') # ['hello', 'world']

m.split.assert_called_with('hello world') # Expected call ; Actual call

m['blah']

m.__getitem__.called # True
m.__getitem__.assert_called_with('blah') # Expected call ; Actual call

'HELLO'

['hello', 'world']

<MagicMock name='mock.__getitem__()' id='2680792450328'>

True

# [assertRaises()]在单元测试中测试异常情况

对于异常的测试可使用 `assertRaises()` 方法。 例如，如果你想测试某个函数抛出了 `ValueError` 异常，像下面这样写：

In [77]:
%%file test.py

import unittest

# A simple function to illustrate
def parse_int(s):
    return int(s)

class TestConversion(unittest.TestCase):
    def test_bad_int(self):
        self.assertRaises(ValueError, parse_int, 'N/A')

Writing test.py


In [78]:
!python -m unittest test.py

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


如果你想测试异常的具体值，需要用到另外一种方法：

In [79]:
%%file test.py
import errno
import unittest

class TestIO(unittest.TestCase):
    def test_file_not_found(self):
        try:
            f = open('/file/not/found')
        except IOError as e:
            self.assertEqual(e.errno, errno.ENOENT)

        else:
            self.fail('IOError not raised')
if __name__ == '__main__':
    unittest.main()

Overwriting test.py


In [80]:
!python -m unittest test.py

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


## 讨论

`assertRaises()` 方法为测试异常存在性提供了一个简便方法。 

一个常见的陷阱是手动去进行异常检测

In [50]:

class TestConversion(unittest.TestCase):
    def test_bad_int(self):
        try:
            r = parse_int('N/A')
        except ValueError as e:
            self.assertEqual(type(e), ValueError)

这种方法的问题在于它很容易遗漏其他情况，比如没有任何异常抛出的时候。 那么你还得需要增加另外的检测过程，如下面这样：

In [ ]:
class TestConversion(unittest.TestCase):
    def test_bad_int(self):
        try:
            r = parse_int('N/A')
        except ValueError as e:
            self.assertEqual(type(e), ValueError)
        else:
            self.fail('ValueError not raised')

`assertRaises()` 方法会处理所有细节，因此你应该使用它。

`assertRaises()` 的一个缺点是它测不了异常具体的值是多少。

为了测试异常值，可以使用 `assertRaisesRegex()` 方法， 它可同时测试异常的存在以及通过正则式匹配异常的字符串表示。例如：

```py
class TestConversion(unittest.TestCase):
    def test_bad_int(self):
        self.assertRaisesRegex(
            ValueError, 
            'invalid literal .*',
            parse_int, 
            'N/A'
        )
```        
        
`assertRaises()` 和 `assertRaisesRegex()` 还有一个容易忽略的地方就是它们还能被当做上下文管理器使用：

```py
class TestConversion(unittest.TestCase):
    def test_bad_int(self):
        with self.assertRaisesRegex(
            ValueError, 
            'invalid literal .*'
            ):
            
            r = parse_int('N/A')
```

但你的测试涉及到多个执行步骤的时候这种方法就很有用了。

# 将测试输出用日志记录到文件中

运行单元测试一个常见技术就是在测试文件底部加入下面这段代码片段：

In [59]:
%%file MyTest.py
import unittest

class MyTest(unittest.TestCase):
    pass

if __name__ == '__main__':
    unittest.main()

Overwriting MyTest.py


In [60]:
!python MyTest.py


----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK


这样的话测试文件就是可执行的，并且会将运行测试的结果打印到标准输出上。 如果你想重定向输出，就需要像下面这样修改 `main()` 函数：

In [61]:
%%file MyTest.py
import unittest
import sys

def main(out=sys.stderr, verbosity=2):
    loader = unittest.TestLoader()
    suite = loader.loadTestsFromModule(sys.modules[__name__])
    unittest.TextTestRunner(out,verbosity=verbosity).run(suite)

if __name__ == '__main__':
    with open('testing.out', 'w') as f:
        main(f)

Overwriting MyTest.py


In [62]:
!python MyTest.py

In [ ]:
# %load testing.out

----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK


`unittest` 模块首先会组装一个测试套件。 

这个测试套件包含了你定义的各种方法。

一旦套件组装完成，它所包含的测试就可以被执行了。

这两步是分开的， `unittest.TestLoader` 实例被用来组装测试套件。

`loadTestsFromModule()` 是它定义的方法之一，用来收集测试用例。 

它会为 `TestCase` 类扫描某个模块并将其中的测试方法提取出来。

如果你想进行细粒度的控制， 可以使用 `loadTestsFromTestCase()` 方法来从某个继承 `TestCase` 的类中提取测试方法。 

`TextTestRunner` 类是一个测试运行类的例子， 这个类的主要用途是执行某个测试套件中包含的测试方法。 

这个类跟执行 `unittest.main()` 函数所使用的测试运行器是一样的。 

不过，我们在这里对它进行了一些列底层配置，包括输出文件和提升级别。

# [@unittest.skip]忽略或期望测试失败

`unittest` 模块有装饰器可用来控制对指定测试方法的处理，例如：

In [69]:
%%file Tests_skip.py
import unittest
import os
import platform

class Tests(unittest.TestCase):
    def test_0(self):
        self.assertTrue(True)

    @unittest.skip('skipped test')
    def test_1(self):
        self.fail('should have failed!')

    @unittest.skipIf(os.name=='posix', 'Not supported on Unix')
    def test_2(self):
        import winreg

    @unittest.skipUnless(platform.system() == 'Darwin', 'Mac specific test')
    def test_3(self):
        self.assertTrue(True)

    @unittest.expectedFailure
    def test_4(self):
        self.assertEqual(2+2, 5)

if __name__ == '__main__':
    unittest.main()

Overwriting Tests_skip.py


In [71]:
!python Tests_skip.py -v

test_0 (__main__.Tests) ... ok
test_1 (__main__.Tests) ... skipped 'skipped test'
test_2 (__main__.Tests) ... ok
test_3 (__main__.Tests) ... skipped 'Mac specific test'
test_4 (__main__.Tests) ... expected failure

----------------------------------------------------------------------
Ran 5 tests in 0.000s

OK (skipped=2, expected failures=1)


如果你在 `Mac` 上运行这段代码，你会得到如下输出：

    bash % python3 testsample.py -v
    test_0 (__main__.Tests) ... ok
    test_1 (__main__.Tests) ... skipped 'skipped test'
    test_2 (__main__.Tests) ... skipped 'Not supported on Unix'
    test_3 (__main__.Tests) ... ok
    test_4 (__main__.Tests) ... expected failure

    ----------------------------------------------------------------------
    Ran 5 tests in 0.002s

    OK (skipped=2, expected failures=1)

`skip()` 装饰器能被用来忽略某个你不想运行的测试。 

`skipIf()` 和 `skipUnless()` 对于你只想在某个特定平台或 `Python` 版本或其他依赖成立时才运行测试的时候非常有用。 

使用 `@expected` 的失败装饰器来标记那些确定会失败的测试，并且对这些测试你不想让测试框架打印更多信息

忽略方法的装饰器还可以被用来装饰整个测试类，比如：

```py
@unittest.skipUnless(platform.system() == 'Darwin', 'Mac specific tests')
class DarwinTests(unittest.TestCase):
    pass
```

# [exception()]处理多个异常

你有一个代码片段可能会抛出多个不同的异常，怎样才能不创建大量重复代码就能处理所有的可能异常呢？

**解决方案**

如果你可以用单个代码块处理不同的异常，可以将它们放入一个元组中，如下所示：

```py
try:
    client_obj.get_url(url)
except (URLError, ValueError, SocketTimeout):
    client_obj.remove_url(url)
```    
    
在这个例子中，元祖中任何一个异常发生时都会执行 `remove_url()` 方法。 如果你想对其中某个异常进行不同的处理，可以将其放入另外一个 `except` 语句中：    
    
```py
try:
    client_obj.get_url(url)
except (URLError, ValueError):
    client_obj.remove_url(url)
except SocketTimeout:
    client_obj.handle_url_timeout(url)
```

很多的异常会有层级关系，对于这种情况，你可能使用它们的一个基类来捕获所有的异常。例如，下面的代码：

```py
try:
    f = open(filename)
except (FileNotFoundError, PermissionError):
    pass
```    
    
可以被重写为：

```py
try:
    f = open(filename)
except OSError:
    pass
```    
    
`OSError` 是 `FileNotFoundError` 和 `PermissionError` 异常的基类。

尽管处理多个异常本身并没什么特殊的，不过你可以使用 `as` 关键字来获得被抛出异常的引用：

```py
try:
    f = open(filename)
except OSError as e:
    if e.errno == errno.ENOENT:
        logger.error('File not found')
    elif e.errno == errno.EACCES:
        logger.error('Permission denied')
    else:
        logger.error('Unexpected error: %d', e.errno)
```

 `e` 变量指向一个被抛出的 `OSError` 异常实例。 
 
这个在你想更进一步分析这个异常的时候会很有用，比如基于某个状态码来处理它。

同时还要注意的时候 `except` 语句是顺序检查的，第一个匹配的会执行。 

你可以很容易的构造多个 `except` 同时匹配的情形，比如：

```py
>>> f = open('missing')
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
FileNotFoundError: [Errno 2] No such file or directory: 'missing'
>>> try:
...     f = open('missing')
... except OSError:
...     print('It failed')
... except FileNotFoundError:
...     print('File not found')
...
It failed
>>>
```

这里的 `FileNotFoundError` 语句并没有执行的原因是 `OSError` 更一般，它可匹配 `FileNotFoundError` 异常， 于是就是第一个匹配的。

在调试的时候，如果你对某个特定异常的类成层级关系不是很确定， 你可以通过查看该异常的 `__mro__` 属性来快速浏览。

比如：

```py
>>> FileNotFoundError.__mro__
(<class 'FileNotFoundError'>, <class 'OSError'>, <class 'Exception'>,
 <class 'BaseException'>, <class 'object'>)
>>>
```

上面列表中任何一个直到 `BaseException` 的类都能被用于 `except` 语句




# [except Exception as e]捕获所有异常

怎样捕获代码中的所有异常？

**解决方案**

想要捕获所有的异常，可以直接捕获 `Exception` 即可：

```py
try:
    ...
except Exception as e:
    ...
    log('Reason:', e)       # Important!
```   
   
这个将会捕获除了 `SystemExit` 、 `KeyboardInterrupt` 和 `GeneratorExit` 之外的所有异常。 

如果你还想捕获这三个异常，将 `Exception` 改成 `BaseException` 即可。



In [89]:
try:
    1/0
except Exception as e:
    print('Reason:', e) 

Reason: division by zero


## 讨论

正因如此，如果你选择捕获所有异常，那么在某个地方（比如日志文件、打印异常到屏幕）打印确切原因就比较重要了。

如果你没有这样做，有时候你看到异常打印时可能摸不着头脑，就像下面这样：

In [82]:
def parse_int(s):
    try:
        n = int(v)
    except Exception:
        print("Couldn't parse")

In [84]:
parse_int('n/a')
parse_int('42')

Couldn't parse
Couldn't parse


像下面这样重写这个函数,这时候你能获取如下输出，指明了有个编程错误：

In [88]:
def parse_int(s):
    try:
        n = int(v)
    except Exception as e:
        print("Couldn't parse")
        print('Reason:', e)
parse_int('42')

Couldn't parse
Reason: name 'v' is not defined


# [繼承Exception]创建自定义异常

创建新的异常很简单——定义新的类，让它继承自 `Exception` （或者是任何一个已存在的异常类型）。 

例如，如果你编写网络相关的程序，你可能会定义一些类似如下的异常：

```py
class NetworkError(Exception):
    pass

class HostnameError(NetworkError):
    pass

class TimeoutError(NetworkError):
    pass

class ProtocolError(NetworkError):
    pass
```    
    
然后用户就可以像通常那样使用这些异常了，例如：

```py
try:
    msg = s.recv()
except TimeoutError as e:
    ...
except ProtocolError as e:
    ...
```    
 
--- 
    
自定义异常类应该总是继承自内置的 `Exception` 类， 或者是继承自那些本身就是从 `Exception` 继承而来的类。 

尽管所有类同时也继承自 `BaseException` ，但你不应该使用这个基类来定义新的异常。 

`BaseException` 是为系统退出异常而保留的，比如 `KeyboardInterrupt` 或 `SystemExit` 以及其他那些会给应用发送信号而退出的异常。 

因此，捕获这些异常本身没什么意义。 

---

这样的话，假如你继承 `BaseException` 可能会导致你的自定义异常不会被捕获而直接发送信号退出程序运行。

在程序中引入自定义异常可以使得你的代码更具可读性，能清晰显示谁应该阅读这个代码。 

还有一种设计是将自定义异常通过继承组合起来。

在复杂应用程序中， 使用基类来分组各种异常类也是很有用的。

它可以让用户捕获一个范围很窄的特定异常，比如下面这样的：

```py
try:
    s.send(msg)
except ProtocolError:
    ...
```    
    
你还能捕获更大范围的异常，就像下面这样：

```py
try:
    s.send(msg)
except NetworkError:
    ...
```    
    
如果你想定义的新异常重写了 `__init__()` 方法， 确保你使用所有参数调用 `Exception.__init__()` ，例如：    
    
```py    
class CustomError(Exception):
    def __init__(self, message, status):
        super().__init__(message, status)
        self.message = message
        self.status = status    
```

不过 `Exception` 的默认行为是接受所有传递的参数并将它们以元组形式存储在 `.args` 属性中. 

很多其他函数库和部分 `Python` 库默认所有异常都必须有 `.args` 属性， 因此如果你忽略了这一步，你会发现有些时候你定义的新异常不会按照期望运行。 

为了演示 `.args` 的使用，考虑下下面这个使用内置的 `RuntimeError` 异常的交互会话， 注意看 `raise` 语句中使用的参数个数是怎样的：



In [91]:
try:
    raise RuntimeError('It failed')
except RuntimeError as e:
    print(e.args)

('It failed',)


In [92]:
try:
    raise RuntimeError('It failed', 42, 'spam')
except RuntimeError as e:

    print(e.args)

('It failed', 42, 'spam')


# [except : raise ]捕获异常后抛出另外的异常

为了链接异常，使用 `raise from` 语句来代替简单的 `raise` 语句。 

它会让你同时保留两个异常的信息。

例如：

def example():
    try:
        int('N/A')
    except ValueError as e:
        raise RuntimeError('A parsing error occurred') from e

example()

在回溯中可以看到，两个异常都被捕获。 

要想捕获这样的异常，你可以使用一个简单的 `except` 语句。 

不过，你还可以通过查看异常对象的 `__cause__` 属性来跟踪异常链。例如：


In [94]:
try:
    example()
except RuntimeError as e:
    print("It didn't work:", e)

    if e.__cause__:
        print('Cause:', e.__cause__)

It didn't work: A parsing error occurred
Cause: invalid literal for int() with base 10: 'N/A'


当在 `except` 块中又有另外的异常被抛出时会导致一个隐藏的异常链的出现。

例如：

In [95]:
def example2():
    try:
        int('N/A')
    except ValueError as e:
        print("Couldn't parse:", err)
        
example2()

NameError: name 'err' is not defined

这个例子中，你同时获得了两个异常的信息，但是对异常的解释不同。 

这时候， `NameError` 异常被作为程序最终异常被抛出，而不是位于解析异常的直接回应中。

如果，你想 **忽略掉异常链** ，可使用 `raise from None` :

In [96]:
def example3():
    try:
        int('N/A')
    except ValueError:
        raise RuntimeError('A parsing error occurred') from None

example3()

RuntimeError: A parsing error occurred

## 讨论

在设计代码时，在另外一个 `except` 代码块中使用 `raise` 语句的时候你要特别小心了。 

大多数情况下，这种 `raise` 语句都应该被改成 `raise from` 语句。

也就是说你应该使用下面这种形式：

```py
try:
   ...
except SomeException as e:
   raise DifferentException() from e
```   
   
这样做的原因是你应该显示的将原因链接起来。 也就是说， `DifferentException` 是直接从 `SomeException` 衍生而来。 这种关系可以从回溯结果中看出来。

如果你像下面这样写代码，你仍然会得到一个链接异常， 不过这个并没有很清晰的说明这个异常链到底是内部异常还是某个未知的编程错误。

```py
try:
   ...
except SomeException:
   raise DifferentException()
```   
   
当你使用 `raise from` 语句的话，就很清楚的表明抛出的是第二个异常。

# 重新抛出被捕获的异常